# Pampers Campaign Automation - Analysis

Junction 2025 - P&G Challenge

**Goal**: Automate personalized marketing campaigns across 20+ countries using AI and SAS Viya

## Approach
1. Connect to SAS Viya
2. Load customer data
3. Exploratory analysis
4. Customer segmentation
5. Campaign targeting

---
## Step 1: Setup and Connection

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

print("✓ Packages loaded")

In [ ]:
# Get OAuth token for Viya
def get_viya_token():
    remote = 'https://viya-i107icluw3.engage.sas.com'
    client_id = os.getenv('VIYA_CLIENT_ID')
    client_secret = os.getenv('VIYA_CLIENT_SECRET')
    
    r = requests.post(
        f'{remote}/SASLogon/oauth/token',
        data='grant_type=client_credentials',
        headers={'Content-Type': 'application/x-www-form-urlencoded'},
        auth=(client_id, client_secret)
    )
    
    if not r.ok:
        raise Exception(f"Auth failed: {r.text}")
    
    return r.json()['access_token']

token = get_viya_token()
print("✓ Viya authentication successful, alhamdulillah")

In [ ]:
# Connect to CAS for analytics
import swat

hostname = os.getenv('VIYA_HOSTNAME')
cas_url = f"https://{hostname}/cas-shared-default-http/"

conn = swat.CAS(cas_url, password=token, ssl_ca_list=False)
print(f"✓ Connected to CAS")
print(f"Session ID: {conn.sessionid}")

---
## Step 2: Create Sample Customer Data

For this hackathon, we'll create realistic sample data representing Pampers customers across multiple countries.

In [ ]:
# Create sample customer data for 20+ countries
np.random.seed(42)

countries = ['US', 'GB', 'DE', 'FR', 'ES', 'IT', 'BR', 'MX', 'CA', 'AU', 
             'JP', 'CN', 'IN', 'NL', 'BE', 'AT', 'CH', 'SE', 'NO', 'DK',
             'PL', 'CZ', 'PT', 'GR', 'TR']

languages = {
    'US': 'en', 'GB': 'en', 'CA': 'en', 'AU': 'en',
    'DE': 'de', 'AT': 'de', 'CH': 'de',
    'FR': 'fr', 'BE': 'fr',
    'ES': 'es', 'MX': 'es',
    'IT': 'it',
    'BR': 'pt', 'PT': 'pt',
    'JP': 'ja',
    'CN': 'zh',
    'IN': 'en',
    'NL': 'nl',
    'SE': 'sv', 'NO': 'no', 'DK': 'da',
    'PL': 'pl', 'CZ': 'cs', 'GR': 'el', 'TR': 'tr'
}

n_customers = 10000

customers = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'country': np.random.choice(countries, n_customers),
    'age': np.random.randint(20, 45, n_customers),
    'days_since_signup': np.random.randint(1, 730, n_customers),
    'total_purchases': np.random.randint(0, 50, n_customers),
    'total_spend': np.random.uniform(0, 1000, n_customers).round(2),
    'email_open_rate': np.random.uniform(0, 1, n_customers).round(3),
    'last_purchase_days': np.random.randint(0, 180, n_customers),
    'baby_age_months': np.random.randint(0, 36, n_customers),
    'preferred_channel': np.random.choice(['email', 'push', 'sms'], n_customers)
})

customers['language'] = customers['country'].map(languages)

print(f"✓ Generated {len(customers):,} customer records")
print(f"✓ Covering {len(countries)} countries")
customers.head()

---
## Step 3: Exploratory Analysis

In [ ]:
# Basic statistics
print("Dataset Overview:")
print(f"Total customers: {len(customers):,}")
print(f"Countries: {customers['country'].nunique()}")
print(f"Languages: {customers['language'].nunique()}")
print(f"\nAverage metrics:")
print(f"  Age: {customers['age'].mean():.1f} years")
print(f"  Total purchases: {customers['total_purchases'].mean():.1f}")
print(f"  Total spend: ${customers['total_spend'].mean():.2f}")
print(f"  Email open rate: {customers['email_open_rate'].mean():.1%}")

In [ ]:
# Country distribution
country_dist = customers['country'].value_counts().head(10)
print("Top 10 countries by customer count:")
print(country_dist)

In [ ]:
# Language distribution
lang_dist = customers['language'].value_counts()
print("Language distribution:")
print(lang_dist)

---
## Step 4: Upload to CAS for Distributed Processing

In [ ]:
# Upload customer data to CAS
print("Uploading data to CAS...")
customers_tbl = conn.upload_frame(
    customers, 
    casout={'name': 'customers', 'replace': True}
)
print("✓ Data uploaded to CAS, alhamdulillah")
print(f"Table: {len(customers_tbl)} rows")

---
## Step 5: Customer Segmentation

Simple customer segmentation based on purchase behavior and engagement.

In [ ]:
# Calculate RFM-style scores for segmentation
customers['recency_score'] = (180 - customers['last_purchase_days']) / 180
customers['frequency_score'] = customers['total_purchases'] / customers['total_purchases'].max()
customers['monetary_score'] = customers['total_spend'] / customers['total_spend'].max()
customers['engagement_score'] = customers['email_open_rate']

# Overall customer value score
customers['value_score'] = (
    customers['recency_score'] * 0.25 +
    customers['frequency_score'] * 0.25 +
    customers['monetary_score'] * 0.3 +
    customers['engagement_score'] * 0.2
).round(3)

print("✓ Customer scores calculated")
print(f"\nValue score distribution:")
print(customers['value_score'].describe())

In [ ]:
# Create segments based on value score
customers['segment'] = pd.cut(
    customers['value_score'],
    bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
    labels=['Low Value', 'Medium-Low', 'Medium', 'Medium-High', 'High Value']
)

print("\nSegment distribution:")
print(customers['segment'].value_counts().sort_index())

print("\nSegment characteristics:")
segment_summary = customers.groupby('segment').agg({
    'customer_id': 'count',
    'total_spend': 'mean',
    'total_purchases': 'mean',
    'email_open_rate': 'mean',
    'value_score': 'mean'
}).round(2)

segment_summary.columns = ['Count', 'Avg_Spend', 'Avg_Purchases', 'Avg_Email_Open', 'Avg_Value_Score']
segment_summary

---
## Step 6: Campaign Targeting

Identify high-value customers per country for targeted campaigns.

In [ ]:
# Select top customers per country (top 20% by value score)
top_customers = customers[customers['value_score'] >= customers['value_score'].quantile(0.8)]

print(f"✓ Selected {len(top_customers):,} high-value customers ({len(top_customers)/len(customers)*100:.1f}%)")
print(f"\nBreakdown by country:")
print(top_customers['country'].value_counts().head(10))

In [ ]:
# Prepare campaign targets by country and language
campaign_targets = top_customers.groupby(['country', 'language']).agg({
    'customer_id': 'count',
    'value_score': 'mean',
    'preferred_channel': lambda x: x.value_counts().index[0],
    'baby_age_months': 'mean'
}).round(2)

campaign_targets.columns = ['Target_Count', 'Avg_Value_Score', 'Primary_Channel', 'Avg_Baby_Age']
campaign_targets = campaign_targets.reset_index()

print("\nCampaign targets by country/language:")
campaign_targets.sort_values('Target_Count', ascending=False)

In [ ]:
# Export top customers for specific country (example: US)
us_targets = top_customers[top_customers['country'] == 'US'].copy()

print(f"\nUS Campaign Targets: {len(us_targets)} customers")
print(f"Avg value score: {us_targets['value_score'].mean():.3f}")
print(f"Total potential spend: ${us_targets['total_spend'].sum():,.2f}")

# Show sample
us_targets[['customer_id', 'value_score', 'total_spend', 'preferred_channel', 'segment']].head(10)

---
## Summary

Alhamdulillah, we've completed:

✓ **Data Processing**: 10,000 customers across 25 countries, 16 languages

✓ **Customer Segmentation**: 5-tier value-based segmentation

✓ **Campaign Targeting**: Top 20% customers identified per country

**For the hackathon demo:**
- Instant processing of multi-country customer data
- Automated segmentation reducing manual work
- Scalable to millions of customers
- Ready for campaign execution via APIs

**Next steps, inshallah:**
1. Build predictive response models
2. Add A/B testing framework
3. Integrate with campaign execution platform
4. Automate daily refresh workflows